<h1> Parameters </h1>

設定変更が必要な項目です。自分の意図した設定になっているか確認してください。

In [1]:
# Global setting
Dict_SetGL = {
    "filname"  : "NB0527", # filter name
    "workdir"  : "../Large_files/mask_fits", # マスク画像の保存先
    "num_seq"  : 1,   # 出力ファイル名につく ID, もし別の設定で実行し直す場合に使う
    "imgdir"   : "../Large_files/imgdata"    # サイエンス画像が配置されている場所
}

# Region file setting
Dict_SetRE = {
    "reg_header" : 3,      # Region ファイルのヘッダの行数 (デフォルトは 3 のはず)
    "cooscale"   : 1.0/0.168 # 画像のピクセルスケール (pix/arcsec の単位の数字を入力)
}

<h1> Modules </h1>

このコードを走らせるために必要なモジュールです。  
ここでエラーが生じる場合は、対応するモジュールが PC にインストールされていません。

In [2]:
import os
import re
import math
import numpy as np

In [3]:
# Pillow
from PIL import Image, ImageDraw

In [4]:
# astronomical packages
from astropy.io import fits
from astropy import wcs
from astropy.modeling.models import Ellipse2D
from astropy.modeling.models import Disk2D
from astropy.modeling.models import Box2D

<h1> Functions </h1>

このノートブックを実行するために必要な関数です。  
自作関数を .py ファイルにコピー&保存して import することも可能です。  
どの時点のバージョンの自作関数を使って計算したのか明確にするため、自分は敢えてノートブックに記載しています。

In [5]:
## Parse and Read ds9 Region file ##
# ds9reg    : ds9 region file in units of wcs degree (default output is recommended)
# num_head  : Row of header in region file (default is 3)
def RegionParser2(ds9reg, num_head):    
    # load region file
    with open(ds9reg) as f:
        l = f.readlines()
    
    arr = []
    for i in range(len(l)):
        if i >= num_head:
            arr.append([i for i in re.split('[,#()"]', l[i].strip()) if not i == ''])
    # 区切り文字は , # ( ) " の 5 つの文字に対応
    # これ以外の文字が入ると正しく配列に格納されないので注意
    
    
    # ndarray として読み込む (Position Angle は elliptical のみ対応)
    # Circle (x_center, y_center, radius)
    ndcirc = np.array( [[tmparr[1],
                         tmparr[2],
                         np.abs(float(tmparr[3]))] 
                        for tmparr in arr if 'circle' in tmparr],
                      dtype=np.float64 )
    
    # Box (x_center, y_center, x_width, y_width)
    ndbox = np.array( [[tmparr[1],
                        tmparr[2],
                        np.abs(float(tmparr[3])),
                        np.abs(float(tmparr[4]))]
                       for tmparr in arr if 'box' in tmparr],
                     dtype=np.float64 )
    
    # Elliptical (x_center, y_center, a_axis, b_axis, theta)
    ndellip = np.array( [[tmparr[1],
                         tmparr[2],
                         np.abs(float(tmparr[3])),
                         np.abs(float(tmparr[4])),
                         tmparr[5]]
                        for tmparr in arr if 'ellipse' in tmparr],
                      dtype=np.float64)
    # Region ファイルを手で作った際, 操作によっては radius 等に負の値が入るので正の値に変換
    
    # Return
    return ndcirc, ndbox, ndellip

In [6]:
## Convert coordinate system ##
# hdu_hdr  :  HDU header of Science Image
# wcirc    :  circle region in units of wcs degree
# wbox     :  box region in units of wcs degree
# wellip   :  elliptical region in units of wcs degree
def CoordW2I(hdu_hdr, wcirc, wbox, wellip):
    # set global setting
    scale_circ  = Dict_SetRE['cooscale']
    scale_box   = Dict_SetRE['cooscale']
    scale_ellip = Dict_SetRE['cooscale']
    # 基本は全て同じ単位になっているはずだが, いちおう領域のタイプごとに変更できるようにした
    
    # load wcs information
    w = wcs.WCS(hdu_hdr)

    # Arrange array
    # (マスク画像作成のために wcirc などの配列を適切な単位にして出力し直す)
    # Circle
    if wcirc.size > 0:
        # wcs -> image の変換 (ds9 coord と python array の Base の違いはここで考慮される)
        ipos_circ = w.wcs_world2pix(wcirc[:,0:2], 0)
        
        # Radius も pixel に変換
        radius_pix = wcirc[:,2] * scale_circ

        # 1つの配列にする
        icirc = np.concatenate([ipos_circ, radius_pix.reshape(-1,1)], 1)
    else:
        icirc = np.array([])

    # Box
    if wbox.size > 0:
        # wcs -> image の変換 (ds9 coord と python array の Base の違いはここで考慮される)
        ipos_box = w.wcs_world2pix(wbox[:,0:2], 0)
        
        # xwid/ywid も pixel に変換
        width_pix = wbox[:,2:4] * scale_box

        # 1つの配列にする
        ibox = np.concatenate([ipos_box, width_pix], 1)
    else:
        ibox = np.array([])


    # Ellipse
    if wellip.size > 0:
        # wcs -> image の変換 (ds9 coord と python array の Base の違いはここで考慮される)
        ipos_ellip = w.wcs_world2pix(wellip[:,0:2], 0)
        
        # a_axis/b_axis も pixel に変換
        axis_pix = wellip[:,2:4] * scale_ellip

        # Position Angle を角度からラジアンに変換
        pa_rad = np.radians(wellip[:,4])
        
        # 1つの配列にする
        iellip = np.concatenate([ipos_ellip, axis_pix, pa_rad.reshape(-1,1)], 1)
    else:
        iellip = np.array([])

    # return
    return icirc, ibox, iellip

In [7]:
## Make masked array ##
# x_imsize :  Image size in X-direction of Science Image
# y_imsize :  Image size in Y-direction of Science Image
# icirc    :  circle region in units of pixel
# ibox     :  box region in units of pixel
# iellip   :  elliptical region in units of pixel
def MaskReg2NdArray(x_imsize, y_imsize, icirc, ibox, iellip):
    
    ## Local Function ##
    # Calculate root sum square
    def CalcRSS(x, y):
        return np.sqrt(x**2.0 + y**2.0)
    
    # Check coordinate (inside or outside of image)
    def ChkReg(ipos, xwid, ywid):
        # 読み込んだサイエンス画像に関係する Region のみ取り出す
        # もし Region が画像に少しでもかかる場合は True / 無関係の場合は False として配列が返される
        # 念のため 5pix 分多めに範囲を広げてチェック
        pos_area = ( (x_imsize+5.0 > ipos[:,0]-xwid) & (ipos[:,0]+xwid > -5.0)
                    & (y_imsize+5.0 > ipos[:,1]-ywid) & (ipos[:,1]+ywid > -5.0) )
        
        return pos_area
        
    
    # set grid (画像の XY 軸と python の配列の対応に注意, 先に Y 軸を入れる)
    y, x = np.mgrid[0:y_imsize, 0:x_imsize]
    
    # 0埋めした配列を用意 (上と同様に XY 軸と配列の対応に注意)
    data = np.zeros( (y_imsize, x_imsize) )
    
    
    # Masking Circular/Box/Elliptical Region
    # Mask した配列を作る (Mask したピクセルには 1 を入れていく)
    # Circle
    if icirc.size > 0:        
        arg_circ = ChkReg(icirc[:,0:2], icirc[:,2], icirc[:,2])
        print(" Total Circluar region    :  {0}".format(np.count_nonzero(arg_circ)))
        
        for coord in icirc[arg_circ]:
            tmp = ( Disk2D(1, coord[0], coord[1], coord[2])(x,y) )
            data = data + tmp
    
    # Box
    if ibox.size > 0:
        arg_box = ChkReg(ibox[:,0:2], ibox[:,2]/2.0, ibox[:,3]/2.0)
        print(" Total Box region         :  {0}".format(np.count_nonzero(arg_box)))
        
        for coord in ibox[arg_box]:
            tmp = ( Box2D(1, coord[0], coord[1], coord[2], coord[3])(x,y) )
            data = data + tmp
    
    # Elliptical
    if iellip.size > 0:
        ellip_xmax = CalcRSS( iellip[:,2] * np.cos(iellip[:,4]),
                              iellip[:,3] * np.sin(iellip[:,4]) )
        ellip_ymax = CalcRSS( iellip[:,2] * np.sin(iellip[:,4]),
                              iellip[:,3] * np.cos(iellip[:,4]) )

        arg_ellip = ChkReg(iellip[:,0:2], ellip_xmax, ellip_ymax)
        print(" Total Elliptical region  :  {}".format(np.count_nonzero(arg_ellip)))
        
        for coord in iellip[arg_ellip]:
            tmp = ( Ellipse2D(1, coord[0], coord[1], coord[2], coord[3], coord[4])(x,y) )
            data = data + tmp
    
    return data

In [8]:
# x_imsize  :  Science image size in X direction in units of pix
# y_imsize  :  Science image size in Y direction in units of pix
# x_ref     :  Reference coodinate of X in units of pix
# x_ref     :  Reference coodinate of Y in units of pix
# idege     :  Coordinates of Edge region
def EdgeReg2NdArray(x_imsize, y_imsize, x_ref, y_ref, iedge):
    
    # Reference 座標から見た Edge Region 座標の角度を調べてソート
    # (Reference は HSC-SSP の画像であれば tract 中心の座標を読み込めば良い)
    # (Pillow/PIL で配列を作る場合, 多角形の頂点は(反)時計回りでソートされている必要があるため)
    degval = [math.degrees(math.atan2(coord[1] - y_ref, coord[0] - x_ref)) for coord in iedge]
    iedge_dsorted = iedge[np.argsort(degval)]
    # わざわざ角度に変換しているが, ラジアンのままでも問題ない
    # (開発時に iedge_dsorted を角度と一緒に出力して目で見て直接確かめたのだが、その時の名残り)
    
    # Pillow (PIL) ではタプルとして与える必要があるので変換
    pos_poly = [(coord[0], coord[1]) for coord in iedge_dsorted]
    
    # Pillow (PIL) を使ってマスク画像の作成
    # (PIL で画像を出力する場合に原点は画像の左上になるが, 配列を作るだけなら問題なし)
    # 画像に対して非常に大きい (例えば patch に対して tract レベルの大きさ) 多角形を入れても問題ない
    img = Image.new('1', (x_imsize, y_imsize), 1)
    ImageDraw.Draw(img).polygon(pos_poly, fill=0)
    mask = np.array(img)
    
    # Return a ndarray mask
    return mask.astype(np.int16)

In [9]:
# inimg      :  Input FITS file
# num_hdu    :  HDU number of science frame
# outimg     :  Output FITS file name
# edgereg    :  Edge Region file
# edge_head  :  Row of header in Edge region file
# maskreg    :  Masked Region file (rejection area)
# mask_head  :  Row of header in Mask region file
# excereg    :  Exception Region file (not rejection area)
# exce_head  :  Row of header in Exception region file
def Mask2FITS(inimg, num_hdu, outimg, **kwargs):
        
    # load Fits image (indicate HDU number by num_hdu)
    hdu = fits.open(inimg)
    sci_header = hdu[num_hdu].header
    hdu.close()
    
    # set data area (XY軸のピクセル数を読み込み)
    img_xsize = sci_header['NAXIS1']
    img_ysize = sci_header['NAXIS2']
    
    # load tract center in units of pixel (Edge area を決めるときに使う)
    tract_xcen = sci_header['CRPIX1']
    tract_ycen = sci_header['CRPIX2']

    if 'edge_cen' in kwargs:
        coord = kwargs['edge_cen']
        tract_xcen = coord[0]
        tract_ycen = coord[1]
    # もし視野の内側を別で与える場合は、変数を上書き
    
    
    # Edge Area
    if 'edgereg' in kwargs:
        print("-- Add Edge Layer")
        wedg_circ, _, _ = RegionParser2( kwargs['edgereg'],
                                         kwargs['edge_head'] )
        iedg_circ, _, _ = CoordW2I( sci_header,
                                    wedg_circ, np.array([]), np.array([]) )        
        # circle 以外は不要なので適当な名前や空の配列を使う
        
        # tract 中心の座標を内側として視野の端を決める
        edge = EdgeReg2NdArray(img_xsize, img_ysize, tract_xcen, tract_ycen, iedg_circ)
    else:
        edge = np.zeros( (img_ysize, img_xsize) )
    
    # Masking Area (Rejection Area)
    if 'maskreg' in kwargs:
        print("-- Add Mask Layer")
        wmsk_circ, wmsk_box, wmsk_ellip = RegionParser2( kwargs['maskreg'],
                                                         kwargs['mask_head'] )
        imsk_circ, imsk_box, imsk_ellip = CoordW2I( sci_header,
                                                    wmsk_circ, wmsk_box, wmsk_ellip )        
        masked = MaskReg2NdArray( img_xsize, img_ysize,
                                  imsk_circ, imsk_box, imsk_ellip )
    else:
        masked = np.zeros( (img_ysize, img_xsize) )
    
    # Exception Area (This area is not masked)
    if 'excereg' in kwargs:
        print("-- Add Exception Layer")
        wexc_circ, wexc_box, wexc_ellip = RegionParser2( kwargs['excereg'],
                                                         kwargs['exce_head'] )
        iexc_circ, iexc_box, iexc_ellip = CoordW2I( sci_header,
                                                    wexc_circ, wexc_box, wexc_ellip )        
        excep = MaskReg2NdArray( img_xsize, img_ysize,
                                 iexc_circ, iexc_box, iexc_ellip )
        excep = np.where(excep > 0, 0, 1)
    else:
        excep = np.ones( (img_ysize, img_xsize) )
    
    
    # Combine all data array
    final_data = np.where((edge + masked) * excep > 0, 1, 0).astype(np.int16)
    
    
    # Make compressed Image HDU
    cHDU = fits.CompImageHDU(final_data, sci_header)
    
    # output
    cHDU.writeto(outimg, overwrite=True)

<h1> Operation Command </h1>

ds9 の Region ファイルから FITS 形式のマスク画像を作成するためのマクロです。  
上記関数をこのマクロ関数で呼び出して実行します。  
入力となる画像ファイルの名前（calexp 変数）などは適宜変更して実行してください。

MacroMF_hsc は HSC の画像に対して実行することを念頭においたマクロになっています。  
HSC 以外の画像について実行する場合は、MacroMF_others を使用してください。  
MacroMF_others では center_edge 変数で視野の内側を定義してやる必要があるので注意してください。

In [10]:
def MacroMF_hsc( tract, px, py ):
    
    # print current patch
    print("\n ======================================================")
    print(" -->> Start creating Mask FITS image for Patch : {0},{1}".format(px, py))
    print(" ======================================================\n")
    
    # general setting
    filname   = Dict_SetGL['filname']
    num_seq   = Dict_SetGL['num_seq']
    
    DIR_img  = Dict_SetGL['imgdir']
    DIR_work = Dict_SetGL['workdir']

    ## Input file setting (自分の用意したファイル名と一致するかチェック!!) ##
    # image / HDU number of science frame
    calexp = os.path.join( DIR_img,
                          'calexp-{0}-{1}-{2},{3}.fits'.format(filname, tract, px, py) )
    hdu_sci = 1
    
    # mask region (マスクする場所を定義した領域ファイル)
    regfile_mask = os.path.join( DIR_work, "maskreg_wcs.reg" )
    header_mask  = Dict_SetRE['reg_header']

    # edge region (視野の端を定義した領域ファイル)
    regfile_edge = os.path.join( DIR_work, "edgereg_wcs.reg" )
    header_edge  = Dict_SetRE['reg_header']
    
    # exception region (例外として mask しない領域を定義した領域ファイル)
    regfile_exce = os.path.join( DIR_work, "excereg_wcs.reg" )
    header_exce  = Dict_SetRE['reg_header']
    
    # output file
    outfits = os.path.join( DIR_work,
                           'Mask-{0}_t{1}p{2}0{3}_{4}.fits'.format(filname, tract, px, py, num_seq) )
    
    
    # check data/directory
    if not os.path.isdir(DIR_work):
        print( "ERROR : Work directory is not found. Check path {0}".format(DIR_work) )
        return
        
    elif not os.path.isfile(calexp):
        print( "Notice : Input image file is not found. Path : {0}".format(calexp))
        print( "-->> Skipped" )
        return

    
    # Check region file path & Make mask FITS image
    if ( os.path.isfile(regfile_mask)
         and os.path.isfile(regfile_edge)
         and os.path.isfile(regfile_exce) ):
        print( "Notice : Use All region files." )
        Mask2FITS( calexp, hdu_sci, outfits,
                   maskreg=regfile_mask, mask_head=header_mask,
                   edgereg=regfile_edge, edge_head=header_edge,
                   excereg=regfile_exce, exce_head=header_exce )
    
    elif ( os.path.isfile(regfile_mask)
           and os.path.isfile(regfile_edge)
           and not os.path.isfile(regfile_exce)):
        print( "Notice : Use Mask and Edge region files." )
        Mask2FITS( calexp, hdu_sci, outfits,
                   maskreg=regfile_mask, mask_head=header_mask,
                   edgereg=regfile_edge, edge_head=header_edge )
        
    elif ( os.path.isfile(regfile_mask)
           and not os.path.isfile(regfile_edge)
           and not os.path.isfile(regfile_exce)):
        print( "Notice : Use Mask region file." )
        Mask2FITS( calexp, hdu_sci, outfits,
                   maskreg=regfile_mask, mask_head=header_mask )
    
    elif ( not os.path.isfile(regfile_mask)
           and os.path.isfile(regfile_edge)
           and not os.path.isfile(regfile_exce)):
        print( "Notice : Use Edge region files." )
        Mask2FITS( calexp, hdu_sci, outfits,
                   edgereg=regfile_edge, edge_head=header_edge )
    
    else:
        print( "Error : All files are not found. Check file path." )
        print( "      : {0}".format(regfile_mask) )
        print( "      : {0}".format(regfile_edge) )
        print( "      : {0}".format(regfile_exce) )

In [11]:
def MacroMF_others( ):
    
    # print current patch
    print( "\n ======================================================" )
    print( " -->> Start creating Mask FITS image " )
    print( " ======================================================\n" )
    
    # general setting
    filname   = Dict_SetGL['filname']
    num_seq   = Dict_SetGL['num_seq']
    
    DIR_img  = Dict_SetGL['imgdir']
    DIR_work = Dict_SetGL['workdir']

    ## Input file setting (自分の用意したファイル名と一致するかチェック!!) ##
    # image / HDU number of science frame
    calexp  = os.path.join( DIR_img, 'calexp-NB0527-9813-4,4.fits' )
    hdu_sci = 1
    
    # mask region (マスクする場所を定義した領域ファイル)
    regfile_mask = os.path.join( DIR_work, "maskreg_wcs.reg" )
    header_mask  = Dict_SetRE['reg_header']

    # edge region (視野の端を定義した領域ファイル)
    regfile_edge = os.path.join( DIR_work, "edgereg_wcs.reg" )
    header_edge  = Dict_SetRE['reg_header']
    center_edge  = [2000, 2200]
    # 視野の内側の座標を image 座標で定義 (この場合 X=2000, Y=2200 を中心として定義)
    
    # exception region (例外として mask しない領域を定義した領域ファイル)
    regfile_exce = os.path.join( DIR_work, "excereg_wcs.reg" )
    header_exce  = Dict_SetRE['reg_header']
    
    # output file
    outfits = os.path.join( DIR_work, 'Mask-NB0527_Img_{0}.fits'.format(num_seq) )
    
    
    # check data/directory
    if not os.path.isdir(DIR_work):
        print( "ERROR : Work directory is not found. Check path {0}".format(DIR_work) )
        return
        
    elif not os.path.isfile(calexp):
        print( "Notice : Input image file is not found. Path : {0}".format(calexp))
        print( "-->> Skipped" )
        return

    
    # Check region file path & Make mask FITS image
    if ( os.path.isfile(regfile_mask)
         and os.path.isfile(regfile_edge)
         and os.path.isfile(regfile_exce) ):
        print( "Notice : Use All region files." )
        Mask2FITS( calexp, hdu_sci, outfits,
                   maskreg=regfile_mask, mask_head=header_mask,
                   edgereg=regfile_edge, edge_head=header_edge, edge_cen=center_edge,
                   excereg=regfile_exce, exce_head=header_exce )
    
    elif ( os.path.isfile(regfile_mask)
           and os.path.isfile(regfile_edge)
           and not os.path.isfile(regfile_exce)):
        print( "Notice : Use Mask and Edge region files." )
        Mask2FITS( calexp, hdu_sci, outfits,
                   maskreg=regfile_mask, mask_head=header_mask,
                   edgereg=regfile_edge, edge_head=header_edge, edge_cen=center_edge )
        
    elif ( os.path.isfile(regfile_mask)
           and not os.path.isfile(regfile_edge)
           and not os.path.isfile(regfile_exce)):
        print( "Notice : Use Mask region file." )
        Mask2FITS( calexp, hdu_sci, outfits,
                   maskreg=regfile_mask, mask_head=header_mask )
    
    elif ( not os.path.isfile(regfile_mask)
           and os.path.isfile(regfile_edge)
           and not os.path.isfile(regfile_exce)):
        print( "Notice : Use Edge region files." )
        Mask2FITS( calexp, hdu_sci, outfits,
                   edgereg=regfile_edge, edge_head=header_edge, edge_cen=center_edge )
    
    else:
        print( "Error : All files are not found. Check file path." )
        print( "      : {0}".format(regfile_mask) )
        print( "      : {0}".format(regfile_edge) )
        print( "      : {0}".format(regfile_exce) )

<h2> Test </h2>

テストで tract = 9813 (COSMOS 領域) の patch = (4,4) と (4,8) に対して実行した結果を示します。  
maskreg_wcs.reg と edgereg_wcs.reg を使った場合、以下のような出力になるはずです。  
今は NB0527 filter に対して実行していますが、他のフィルターを使ったとしても同じ出力になります。

In [12]:
# test run
MacroMF_hsc( 9813, 4, 4 )


 -->> Start creating Mask FITS image for Patch : 4,4

Notice : Use Mask and Edge region files.
-- Add Edge Layer
-- Add Mask Layer
 Total Circluar region    :  4
 Total Box region         :  2
 Total Elliptical region  :  2


In [13]:
# test run
MacroMF_hsc( 9813, 4, 8 )


 -->> Start creating Mask FITS image for Patch : 4,8

Notice : Use Mask and Edge region files.
-- Add Edge Layer
-- Add Mask Layer
 Total Circluar region    :  0
 Total Box region         :  0
 Total Elliptical region  :  0


In [14]:
# test run
MacroMF_others( )


 -->> Start creating Mask FITS image 

Notice : Use Mask and Edge region files.
-- Add Edge Layer
-- Add Mask Layer
 Total Circluar region    :  4
 Total Box region         :  2
 Total Elliptical region  :  2


<h1> Run </h1>

HSC で観測された撮像データに一括で実行する場合の一例。

<h3> Patch = 2,4 - 8,4 </h3>

In [ ]:
#for x in range(2, 9, 1):
#    MacroMF_hsc(9813, x, 4)

実行するとノートブックのサイズが大きくなりすぎてしまうので、コメントアウトしています。  
ファイルを適切に配置すれば実行できるはずです。